#### Imports

In [1]:
import pandas as pd 
import re
from pathlib import Path
import create_dataset as cd
import feature_engineering as fe
import os 
from prep import * 

In [2]:
data_tim = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_tim_copy.csv')
data_sien = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_sien_copy.csv')

In [19]:
data = cd.combine_start_dfs(data_sien, data_tim)
# change name climber to numerical value
data['name_climber'] = data['name_climber'].map({'sien':0,'tim':1})
# drop weird column (SIEN's FAULT)
data = data.drop(columns = ['Time (s).1'])
# drop climber rating var 
data = cd.drop_climber_rating(data)
# add datetime var 
data = cd.add_datetime(data)
# add heart-rate var
# TODO
# cut falls 
data = cd.cut_fall(data)

data = fe.features(data)
data.grading = data.grading.apply(fe.french_grade_to_num)

data = create_snippets(data, 20, 5)
data = change_climb_ID(data)

data

/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/prep.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_20sec.loc[:, "snippet"] = "random_snippet"
/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/prep.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_20sec.loc[:, "fall_top"] = 0
/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/prep.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,Time (s),X (m/s^2),Y (m/s^2),Z (m/s^2),X (rad/s),Y (rad/s),Z (rad/s),name_climber,grading,num_attempt,...,roll,pitch,Velocity_X,Velocity_Y,Velocity_Z,Displacement_X,Displacement_Y,Displacement_Z,distance,climb_id
0,108.088635,0.157283,-0.442686,-1.072861,0.039662,-0.432147,-0.123366,0,4.3,1,...,-2.750253,0.134698,106.851852,75.431705,-75.699002,737.175067,-2137.185487,-1975.120968,3002.014611,1.1
1,108.289898,-0.659116,0.043920,-0.302016,-0.675243,-0.943927,-0.006866,0,4.3,1,...,2.997183,-1.137157,106.718612,75.440132,-75.759373,758.649546,-2121.990499,-1990.357732,3006.645425,1.1
2,108.491162,1.083198,-0.937163,0.811129,-0.707245,-0.285354,-0.007537,0,4.3,1,...,-0.857363,0.718231,106.937151,75.251890,-75.596498,780.175914,-2106.855546,-2005.582475,3011.623451,1.1
3,108.692425,0.330704,0.027100,0.293211,-0.433973,-0.436695,-0.142976,0,4.3,1,...,0.092163,0.843308,107.003178,75.256970,-75.537110,801.707818,-2091.698635,-2020.775335,3016.864524,1.1
4,108.893689,-0.816926,0.448517,-0.063428,-0.500497,-0.702771,0.051526,0,4.3,1,...,1.711283,-1.064510,106.839292,75.347614,-75.550251,823.214705,-2076.544265,-2035.990921,3022.412541,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,65.444887,0.187599,0.529382,-0.260925,-0.252542,0.172883,0.150463,0,6.1,3,...,2.028737,0.307761,15.209993,10.032655,-29.850457,1116.856019,486.710326,481.178364,1309.880501,25.2
4996,65.646174,0.217285,0.427087,0.223965,-0.195443,0.078290,0.001839,0,6.1,3,...,1.087820,0.423325,15.253626,10.118554,-29.805173,1119.918774,488.743746,475.175695,1311.060812,25.2
4997,65.847461,0.751633,0.195774,0.034231,-0.330692,0.036803,0.180573,0,6.1,3,...,1.397697,1.312297,15.404920,10.157960,-29.798283,1123.019584,490.788412,469.177688,1312.316256,25.2
4998,66.048749,0.862736,0.068156,0.200585,-0.349219,0.154453,0.084462,0,6.1,3,...,0.327548,1.330006,15.578655,10.171730,-29.758055,1126.160960,492.838297,463.190080,1313.648790,25.2


In [23]:
aggregation_method = {} 
sum_vars = ['Time (s)', 'distance'] 
max_vars = ['climb_id','name_climber', 'num_attempt', 'fall_top']


for var in data.columns: 
    if var in sum_vars: 
        aggregation_method[var] = 'sum' 
    elif var in max_vars:
        aggregation_method[var] = 'max'
    else: 
        aggregation_method[var] = 'mean' 

aggregation_method

data.dtypes

grouped = data.groupby('climb_id').agg(aggregation_method)

grouped

,Time (s),X (m/s^2),Y (m/s^2),Z (m/s^2),X (rad/s),Y (rad/s),Z (rad/s),name_climber,grading,num_attempt,...,roll,pitch,Velocity_X,Velocity_Y,Velocity_Z,Displacement_X,Displacement_Y,Displacement_Z,distance,climb_id
climb_id,,,,,,,,,,,,,,,,,,,,,
1.1,11805.117282,0.073643,-0.114770,-0.020949,-0.034898,-0.014177,-0.023187,0,4.3,1,...,-0.324843,0.061606,107.467035,74.480116,-75.658128,1805.750491,-1391.159539,-2727.817362,3.631075e+05,1.1
1.2,5123.673621,0.063358,0.030584,-0.040891,-0.018469,-0.022327,0.020524,0,4.3,1,...,0.202381,0.019436,105.987352,76.853138,-74.411494,-5341.697071,-6521.766872,2307.227968,8.832784e+05,1.2
10.1,19513.470416,0.084181,-0.021301,-0.081466,-0.002549,0.060805,0.045960,1,6.1,1,...,-0.148235,0.032207,117.046759,97.021745,-130.398271,13857.479100,8676.963228,-17615.594012,2.187142e+06,10.1
10.2,16812.135436,0.044234,0.016318,-0.083228,0.007590,0.015897,0.007138,1,6.1,1,...,0.175901,-0.024387,106.403763,88.561642,-118.780195,7495.694424,3623.865155,-10371.236377,1.343534e+06,10.2
11.1,8414.810701,-0.039097,0.028289,-0.031055,-0.038858,0.017670,0.023514,1,5.4,1,...,0.221656,-0.027411,98.588057,77.678062,-102.375406,-1512.497296,-3567.927206,-758.911685,4.028557e+05,11.1
11.2,1413.720741,0.099969,-0.053627,-0.153520,-0.021870,0.028730,-0.003542,1,5.4,1,...,-0.008652,0.070094,95.425253,77.249355,-97.677944,-8347.879928,-8980.241896,6335.388592,1.394066e+06,11.2
12.1,21870.228733,-0.009154,0.048165,-0.009186,-0.051580,-0.017347,-0.025414,0,6.0,2,...,-0.055875,-0.000941,39.928199,38.332174,-52.870594,5988.582799,5010.606121,-7538.245176,1.085334e+06,12.1
12.2,15796.629727,0.090883,-0.019684,-0.008683,-0.017194,-0.031656,0.022693,0,6.0,2,...,0.059159,-0.021774,39.868936,37.412255,-51.147779,3488.196940,2701.746113,-4303.932676,6.225385e+05,12.2
13.1,5864.632896,0.086263,0.063388,0.015433,-0.026371,-0.050674,-0.003363,0,6.4,1,...,0.051891,0.032284,40.413165,32.319301,-46.984603,-539.477447,-1002.001266,461.894064,1.245212e+05,13.1


#### Add datetime to df

Get datetime from phyphox metadata

In [4]:
raw_directory = Path('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/climb_data_phyphox')

# subdirectories 
subdir_sien = raw_directory.joinpath('sien')
subdir_tim = raw_directory.joinpath('tim')

# datetime 
datetime_dict = {}

# get start & end datetimes for each climb entry 
for climber in (subdir_sien, subdir_tim):
    for file_path in climber.iterdir():

        if file_path.name == '.DS_Store': continue 
        # get climb entry number
        entry_num = int(re.findall(r'\d+', file_path.name)[0])
        # dirs for each climb
        climbdir = raw_directory.joinpath(file_path)
        # metadata dir 
        metadir = climbdir.joinpath('meta')
        # open time file 
        with open(metadir.joinpath('time.csv'), 'r') as datetime_file:
            # put start & end time into dict 
            datetime_df = pd.read_csv(datetime_file)
            start_time, end_time = datetime_df['system time text'][0], datetime_df['system time text'][1]
            datetime_dict[entry_num] = (start_time, end_time)

print(datetime_dict)

{15: ('2023-06-07 16:36:31.299 UTC+02:00', '2023-06-07 16:38:32.531 UTC+02:00'), 4: ('2023-06-07 13:45:35.533 UTC+02:00', '2023-06-07 13:48:47.258 UTC+02:00'), 5: ('2023-06-07 13:53:48.058 UTC+02:00', '2023-06-07 13:56:49.460 UTC+02:00'), 14: ('2023-06-07 16:31:16.179 UTC+02:00', '2023-06-07 16:32:59.567 UTC+02:00'), 25: ('2023-06-07 18:05:25.347 UTC+02:00', '2023-06-07 18:08:21.544 UTC+02:00'), 2: ('2023-06-07 13:28:35.606 UTC+02:00', '2023-06-07 13:30:21.669 UTC+02:00'), 13: ('2023-06-07 16:28:15.035 UTC+02:00', '2023-06-07 16:29:26.392 UTC+02:00'), 12: ('2023-06-07 16:18:55.545 UTC+02:00', '2023-06-07 16:22:44.271 UTC+02:00'), 3: ('2023-06-07 13:37:23.658 UTC+02:00', '2023-06-07 13:39:58.248 UTC+02:00'), 24: ('2023-06-07 17:58:33.461 UTC+02:00', '2023-06-07 18:03:18.690 UTC+02:00'), 1: ('2023-06-07 13:12:51.871 UTC+02:00', '2023-06-07 13:15:00.053 UTC+02:00'), 17: ('2023-06-07 16:57:12.188 UTC+02:00', '2023-06-07 17:01:05.538 UTC+02:00'), 16: ('2023-06-07 16:43:36.219 UTC+02:00', '2

In [5]:

# creates & maps datetime sequences for each climb entry 
datetime_sequence_dict = {}
for group_name, group_data in data.groupby('entry_num'):
    # create & save datetime sequence for each climb entry 
    start_datetime, end_datetime = pd.to_datetime(datetime_dict[group_name][0]), pd.to_datetime(datetime_dict[group_name][1])
    datetime_range = pd.date_range(start=start_datetime, end=end_datetime, periods=len(group_data))
    datetime_sequence_dict[group_name] = datetime_range

# inits null datetime var
if not 'datetime' in data:
    data.insert(1, 'datetime', float('nan'))

# fills in datetime sequences 
for entry_id in data['entry_num'].unique():
    mask = data['entry_num'] == entry_id
    data.loc[mask, 'datetime'] = datetime_sequence_dict[entry_id]

# check 
data['datetime'].isna().sum()


0

#### cutting falls from data

#### EDA - table

In [10]:
# code from DMT
# datatypes - NOTE: subject to change (only accelerometer & gyrometer rn)
types = ['id','datetime', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'categorical', 'ordinal', ] 

# nb of values 
n_values = [data[var].count() for var in data]

# nb of unique values
n_unique = [data[var].nunique() for var in data]


# TODO: do once we have the final variables (eg. for grade)
## mean + std 
# mean_std = []

# for var in data: 
#     if var not in ['datetime', 'entry_num']: # TODO 
#         mean = round(data[var].mean(), 2)
#         std = round(data[var].std(), 2) 
#         res = f'{mean} ({std})'
#         mean_std.append(res)
# mean_std.insert(0,'Na')
# mean_std.insert(0,'Na')

# # range of values
# val_range = []
# for var in data:
#     if var != 'id':
#         r = (data[var].min(), data[var].max())
#         if var != 'time':
#             r = (round(r[0]), round(r[1]))
#         else:
#             r = (data['time'].dt.date.min(), data['time'].dt.date.max())
#         val_range.append(r)
#         r = data['time'].dt.date
# val_range.insert(0, 'Na')

# # date range 
# time_range = []
# for var in data: 
#     # if var != 'id':
#     grouped = data.groupby(var)['time'].min()
#     r = (grouped.dt.date.min(), grouped.dt.date.max())
#     time_range.append(r)


# # missing values before time aggregation TODO: do for heartrate only ig
# missing_before_agg = data.isna().sum().tolist()

# # missing values after time aggregation -> fill this in manually... 
# # automatic 14 for each except call & sms 
# # + 13 for activity 
# # + 1 for 'circumplex.arousal','circumplex.valence', 'mood' & -2 cuz there were like 2 days of measurements in the first 14 days? 
# # temp_data = fn.group_data(data, count=False)
# # missing_after_agg = temp_data.isna().sum().tolist()
# # missing_after_agg.insert(0, 0)


# missing_after_agg = [0 for i in range(len(data.columns))]
# for i, var in enumerate(data.columns):
#     if var == 'sms' or var == 'call' or var == 'id' or var == 'time': 
#         continue
#     if var == 'circumplex.arousal' or var == 'circumplex.valence' or var == 'mood':
#         missing_after_agg[i] += 1
#     missing_after_agg[i] += 14
#     if var == 'activity':
#         missing_after_agg[i] += 13



data_table = pd.DataFrame({'Variables': data.columns, 
                    # 'Data types': types, 
                    'number values': n_values, 
                    'number unique values': n_unique, 
                    # 'Mean (std)': mean_std,
                    # 'Value range': val_range,
                    # 'Date range': time_range, 
                    # 'Missing values (before time aggregation)': missing_before_agg,
                    # 'Missing values (after time aggregation)': missing_after_agg
                    })
data_table

,Variables,number values,number unique values
0,entry_num,18134,25
1,datetime,18134,18134
2,Time (s),18134,18134
3,X (m/s^2),18134,18131
4,Y (m/s^2),18134,18086
5,Z (m/s^2),18134,18111
6,Time (s).1,18134,18134
7,X (rad/s),18134,18133
8,Y (rad/s),18134,18134
9,Z (rad/s),18134,18132
